In [141]:
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import glob
import parselmouth
from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.io import wavfile
from scipy.signal import stft
import nolds
import numba as nb
from scipy.stats import entropy
from typing import Optional, Tuple

# MFCCs 

In [142]:
f = pd.read_csv('data-f.csv')


In [143]:
for i in f['id']:
    # load audio files with librosa
    signal, sr = librosa.load('audiodata/'+str(i))
    mfccs = np.mean(librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sr).T,axis=0)
    delta_mfccs = np.mean(librosa.feature.delta(mfccs).T,axis=0)
    delta2_mfccs = np.mean(librosa.feature.delta(mfccs, order=2).T,axis=0)
    f.loc[f['id'] == i, 'MFCC-1'] =mfccs[0]
    f.loc[f['id'] == i, 'MFCC-2'] =mfccs[1]
    f.loc[f['id'] == i, 'MFCC-3'] =mfccs[2]
    f.loc[f['id'] == i, 'MFCC-4'] =mfccs[3]
    f.loc[f['id'] == i, 'MFCC-5'] =mfccs[4]
    f.loc[f['id'] == i, 'MFCC-6'] =mfccs[5]
    f.loc[f['id'] == i, 'MFCC-7'] =mfccs[6]
    f.loc[f['id'] == i, 'MFCC-8'] =mfccs[7] 
    f.loc[f['id'] == i, 'MFCC-9'] =mfccs[8]
    f.loc[f['id'] == i, 'MFCC-10'] =mfccs[9]
    f.loc[f['id'] == i, 'MFCC-11'] =mfccs[10]
    f.loc[f['id'] == i, 'MFCC-12'] =mfccs[11]
    f.loc[f['id'] == i, 'MFCC-13'] =mfccs[12]
    f.loc[f['id'] == i, '1st-derivative'] =delta_mfccs
    f.loc[f['id'] == i, '2nd-derivative'] =delta2_mfccs

In [144]:
f

,id,gender,age,status,MFCC-1,MFCC-2,MFCC-3,MFCC-4,MFCC-5,MFCC-6,MFCC-7,MFCC-8,MFCC-9,MFCC-10,MFCC-11,MFCC-12,MFCC-13,1st-derivative,2nd-derivative
0,1-a_n.wav,female,17-29,healthy,-229.900299,70.987656,-31.600691,-38.037273,-37.097298,-28.244047,23.086527,-3.702321,-29.035618,19.684288,-11.513266,-2.432917,-5.764521,6.110204,-4.813137
1,10-a_n.wav,female,17-29,healthy,-177.231812,124.980606,-4.989427,-26.419960,-10.432103,5.199066,-13.459819,29.368608,-18.531691,6.474063,-0.609437,13.707862,0.313944,2.660672,-2.824473
2,1059-a_n.wav,female,17-29,healthy,-215.689453,132.913712,-38.871655,-42.621090,-27.748386,-14.176871,-2.757598,23.727890,-22.948891,6.631990,9.736770,-17.351088,-0.563084,4.590938,-3.732108
3,1060-a_n.wav,female,17-29,healthy,-179.004013,137.124176,-41.186527,-12.956992,-23.057871,1.249742,12.310613,5.677735,-22.362137,22.160065,-19.430182,9.689588,-30.554930,1.903684,-3.933841
4,1065-a_n.wav,female,17-29,healthy,-219.231903,118.439713,-53.232697,-12.397227,-0.406237,1.629000,-0.372281,1.487446,-34.674034,20.461397,-3.294332,-12.353028,-6.538914,2.801241,-4.350231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,916-a_n.wav,male,60+,pathological,-206.275269,199.477036,-16.151987,-5.054139,-28.327045,-15.628398,-3.559500,26.115755,7.507662,-8.994985,8.004864,-2.805704,13.759897,3.060516,-2.444826
1360,918-a_n.wav,male,60+,pathological,-83.471779,134.779114,-53.661423,-13.109919,-63.956722,1.597356,-24.768343,9.757913,-19.314672,-2.862256,13.001028,0.785078,-13.930746,1.780056,-0.592388
1361,922-a_n.wav,male,60+,pathological,-281.555756,168.036407,28.660950,-7.232946,-38.963890,-7.814531,-7.640507,17.536627,-7.195324,9.665165,-6.795200,7.274139,24.941833,4.954917,-4.336839
1362,923-a_n.wav,male,60+,pathological,-218.406387,170.818329,-12.190666,-3.343062,-26.285761,-4.524966,17.202215,-0.297808,3.650085,-4.840356,-6.634346,19.276258,4.910005,3.197191,-3.278336


In [145]:
from sklearn.preprocessing import MinMaxScaler

# select the columns to scale
cols_to_scale = ['MFCC-1', 'MFCC-2', 'MFCC-3', 'MFCC-4', 'MFCC-5', 'MFCC-6', 'MFCC-7', 'MFCC-8', 'MFCC-9', 'MFCC-10', 'MFCC-11', 'MFCC-12', 'MFCC-13', '1st-derivative', '2nd-derivative']

# create a scaler object
scaler = MinMaxScaler(feature_range=(-1, 1))

# fit and transform the selected columns
scaled_cols = scaler.fit_transform(f[cols_to_scale])

# create a new dataframe with the scaled columns
df_scaled = pd.DataFrame(scaled_cols, columns=cols_to_scale)

# replace the original columns with the scaled ones
f[cols_to_scale] = df_scaled[cols_to_scale]


In [146]:
f

,id,gender,age,status,MFCC-1,MFCC-2,MFCC-3,MFCC-4,MFCC-5,MFCC-6,MFCC-7,MFCC-8,MFCC-9,MFCC-10,MFCC-11,MFCC-12,MFCC-13,1st-derivative,2nd-derivative
0,1-a_n.wav,female,17-29,healthy,-0.129712,-0.823662,-0.218797,-0.560799,-0.135607,-0.297418,0.534995,-0.186073,-0.530086,0.478073,-0.182316,-0.113946,0.080308,0.845591,-0.127952
1,10-a_n.wav,female,17-29,healthy,0.199122,-0.206554,0.130936,-0.352918,0.412921,0.309148,-0.284263,0.525699,-0.261176,0.151716,0.124619,0.314070,0.228501,-0.001756,0.332969
2,1059-a_n.wav,female,17-29,healthy,-0.040987,-0.115883,-0.314355,-0.642822,0.056709,-0.042278,-0.044352,0.404296,-0.374261,0.155618,0.415858,-0.509541,0.207119,0.472397,0.122602
3,1060-a_n.wav,female,17-29,healthy,0.188057,-0.067760,-0.344777,-0.112011,0.153197,0.237518,0.293432,0.015810,-0.359239,0.539237,-0.405171,0.207515,-0.524082,-0.187704,0.075846
4,1065-a_n.wav,female,17-29,healthy,-0.063104,-0.281313,-0.503092,-0.101994,0.619163,0.244397,0.009120,-0.074376,-0.674435,0.497271,0.049041,-0.377004,0.061428,0.032773,-0.020662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,916-a_n.wav,male,60+,pathological,0.017790,0.644897,-0.015766,0.029404,0.044806,-0.068605,-0.062328,0.455689,0.405454,-0.230444,0.367106,-0.123831,0.556313,0.096462,0.420961
1360,918-a_n.wav,male,60+,pathological,0.784509,-0.094563,-0.508726,-0.114747,-0.688131,0.243823,-0.537766,0.103626,-0.281222,-0.078936,0.507744,-0.028612,-0.118785,-0.218072,0.850308
1361,922-a_n.wav,male,60+,pathological,-0.452221,0.285549,0.573179,-0.009584,-0.174004,0.073117,-0.153812,0.271044,0.029045,0.230552,-0.049506,0.143463,0.828928,0.561805,-0.017558
1362,923-a_n.wav,male,60+,pathological,-0.057950,0.317345,0.036295,0.060022,0.086797,0.132781,0.403087,-0.112799,0.306697,-0.127805,-0.044978,0.461731,0.340552,0.130035,0.227775


# Fo-Jitter-Shimmer-Hnr

In [147]:
# This is the function to measure voice pitch
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer


In [148]:
# create lists to put the results
file_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []

# Go through all the wave files in the folder and measure pitch
for wave_file in glob.glob("audiodata/*.wav"):
    sound = parselmouth.Sound(wave_file)
    (meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 500, "Hertz")
    file_list.append(wave_file) # make an ID list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr)
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
df = pd.DataFrame(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list]),
                               columns=['id', 'meanF0Hz', 'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                        'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                        'apq11Shimmer', 'ddaShimmer'])  #add these lists to pandas in the right order







In [149]:
df

,id,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
0,audiodata\1-a_n.wav,274.4599069178484,2.335414547808669,23.686922944564888,0.0028342307015571293,1.0326599888483383e-05,0.0016500755384055502,0.0015606764040807465,0.0049502266152166505,0.033892549243638034,0.29473439843174226,0.020369088536211503,0.019883596349730193,0.021808587713530222,0.06110726560863451
1,audiodata\10-a_n.wav,201.01582906584565,1.6018000765617861,21.394700330323666,0.0031729426861521704,1.5784922922050765e-05,0.0017956372972826398,0.001972168759744504,0.00538691189184792,0.03171176465731839,0.28050325305763135,0.013830946439636671,0.020864517898700755,0.038690969455642966,0.04149283931891001
2,audiodata\100-a_n.wav,113.5683911609963,0.9237358917079987,23.618812979830683,0.003343030392475076,2.9433525620860033e-05,0.001779778604858804,0.0021429019565045917,0.005339335814576412,0.0522823842469888,0.4548592512843616,0.02878448879477906,0.03028503863664831,0.038011429753532666,0.08635346638433718
3,audiodata\1000-a_n.wav,106.34449346924008,3.1308214542886996,21.64571510512745,0.00600935122781834,5.650297986732578e-05,0.0023065122680897684,0.0036184509511001596,0.006919536804269305,0.03164888064139259,0.28074480985301226,0.016428857101280808,0.019516091810544894,0.027407318083085438,0.049286571303842425
4,audiodata\1002-a_n.wav,88.49746894838327,1.028905337186477,23.212946788788113,0.005422522601674777,6.127448063406365e-05,0.002852649408030298,0.003167324096474038,0.008557948224090894,0.021790858445165734,0.18990910339144265,0.010950963096697018,0.013614199845358095,0.02080181961774985,0.03285288929009105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,audiodata\995-a_n.wav,103.06175743160858,1.3548600146492609,15.480068917429307,0.008106787620262164,7.867273972168569e-05,0.0050922653516523924,0.0043820653408013915,0.015276796054957176,0.21553646509697535,1.7807225812179905,0.14057807432872094,0.12680976038139066,0.14238126133844997,0.4217342229861628
1360,audiodata\996-a_n.wav,125.19165140960098,1.554584286457678,20.663719289635623,0.0082722373478068,6.608768292773141e-05,0.004419795134291679,0.004859374677441282,0.013259385402875037,0.023456711762093633,0.2142129530380046,0.008830502908275363,0.01258726107155404,0.026272649869177395,0.026491508724826092
1361,audiodata\997-a_n.wav,101.33283145616501,0.8873447575839802,21.6274089960974,0.00358125395124866,3.534171528158072e-05,0.0017927701511538821,0.002113998896588684,0.0053783104534616466,0.02527187907022999,0.21678783028103496,0.008554683711090345,0.016186276410574192,0.028928264982986097,0.025664051133271033
1362,audiodata\998-a_n.wav,143.97126159463073,1.9345601222997566,30.510630551761963,0.0019324362784195602,1.3421898167609635e-05,0.0010268173423345678,0.001196583954299889,0.003080452027003703,0.01658553217579049,0.1376586847391362,0.008234303484256739,0.01004295755976886,0.013603169212613519,0.02470291045277022


In [150]:
df['id'] = df['id'].str.replace(("audiodata"), '' )
df['id'] = df['id'].str.replace(("\\"), '' )

In [151]:
df

,id,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
0,1-a_n.wav,274.4599069178484,2.335414547808669,23.686922944564888,0.0028342307015571293,1.0326599888483383e-05,0.0016500755384055502,0.0015606764040807465,0.0049502266152166505,0.033892549243638034,0.29473439843174226,0.020369088536211503,0.019883596349730193,0.021808587713530222,0.06110726560863451
1,10-a_n.wav,201.01582906584565,1.6018000765617861,21.394700330323666,0.0031729426861521704,1.5784922922050765e-05,0.0017956372972826398,0.001972168759744504,0.00538691189184792,0.03171176465731839,0.28050325305763135,0.013830946439636671,0.020864517898700755,0.038690969455642966,0.04149283931891001
2,100-a_n.wav,113.5683911609963,0.9237358917079987,23.618812979830683,0.003343030392475076,2.9433525620860033e-05,0.001779778604858804,0.0021429019565045917,0.005339335814576412,0.0522823842469888,0.4548592512843616,0.02878448879477906,0.03028503863664831,0.038011429753532666,0.08635346638433718
3,1000-a_n.wav,106.34449346924008,3.1308214542886996,21.64571510512745,0.00600935122781834,5.650297986732578e-05,0.0023065122680897684,0.0036184509511001596,0.006919536804269305,0.03164888064139259,0.28074480985301226,0.016428857101280808,0.019516091810544894,0.027407318083085438,0.049286571303842425
4,1002-a_n.wav,88.49746894838327,1.028905337186477,23.212946788788113,0.005422522601674777,6.127448063406365e-05,0.002852649408030298,0.003167324096474038,0.008557948224090894,0.021790858445165734,0.18990910339144265,0.010950963096697018,0.013614199845358095,0.02080181961774985,0.03285288929009105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,995-a_n.wav,103.06175743160858,1.3548600146492609,15.480068917429307,0.008106787620262164,7.867273972168569e-05,0.0050922653516523924,0.0043820653408013915,0.015276796054957176,0.21553646509697535,1.7807225812179905,0.14057807432872094,0.12680976038139066,0.14238126133844997,0.4217342229861628
1360,996-a_n.wav,125.19165140960098,1.554584286457678,20.663719289635623,0.0082722373478068,6.608768292773141e-05,0.004419795134291679,0.004859374677441282,0.013259385402875037,0.023456711762093633,0.2142129530380046,0.008830502908275363,0.01258726107155404,0.026272649869177395,0.026491508724826092
1361,997-a_n.wav,101.33283145616501,0.8873447575839802,21.6274089960974,0.00358125395124866,3.534171528158072e-05,0.0017927701511538821,0.002113998896588684,0.0053783104534616466,0.02527187907022999,0.21678783028103496,0.008554683711090345,0.016186276410574192,0.028928264982986097,0.025664051133271033
1362,998-a_n.wav,143.97126159463073,1.9345601222997566,30.510630551761963,0.0019324362784195602,1.3421898167609635e-05,0.0010268173423345678,0.001196583954299889,0.003080452027003703,0.01658553217579049,0.1376586847391362,0.008234303484256739,0.01004295755976886,0.013603169212613519,0.02470291045277022


In [152]:
df.isnull().values.any()

False

In [153]:
f = pd.merge(f, df, on = 'id')

In [154]:
f

,id,gender,age,status,MFCC-1,MFCC-2,MFCC-3,MFCC-4,MFCC-5,MFCC-6,...,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
0,1-a_n.wav,female,17-29,healthy,-0.129712,-0.823662,-0.218797,-0.560799,-0.135607,-0.297418,...,1.0326599888483383e-05,0.0016500755384055502,0.0015606764040807465,0.0049502266152166505,0.033892549243638034,0.29473439843174226,0.020369088536211503,0.019883596349730193,0.021808587713530222,0.06110726560863451
1,10-a_n.wav,female,17-29,healthy,0.199122,-0.206554,0.130936,-0.352918,0.412921,0.309148,...,1.5784922922050765e-05,0.0017956372972826398,0.001972168759744504,0.00538691189184792,0.03171176465731839,0.28050325305763135,0.013830946439636671,0.020864517898700755,0.038690969455642966,0.04149283931891001
2,1059-a_n.wav,female,17-29,healthy,-0.040987,-0.115883,-0.314355,-0.642822,0.056709,-0.042278,...,2.1295956656440397e-05,0.0030634005468321464,0.0030185622087256837,0.00919020164049644,0.02560829590679049,0.22208931564625167,0.014751893970506413,0.016123421797386565,0.016943343522295626,0.04425568191151924
3,1060-a_n.wav,female,17-29,healthy,0.188057,-0.067760,-0.344777,-0.112011,0.153197,0.237518,...,2.7232999135153376e-05,0.0036924229757285803,0.0031757959783733614,0.011077268927185741,0.04588122600839834,0.3991748537487175,0.02829794510054568,0.023999097612021354,0.03147154705700457,0.08489383530163705
4,1065-a_n.wav,female,17-29,healthy,-0.063104,-0.281313,-0.503092,-0.101994,0.619163,0.244397,...,1.0867775002464293e-05,0.001552792826095096,0.0015809278832064706,0.0046583784782852885,0.02605246109627153,0.22654225245114276,0.013355474757638522,0.014128205939053248,0.02412114213390821,0.04006642427291557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,916-a_n.wav,male,60+,pathological,0.017790,0.644897,-0.015766,0.029404,0.044806,-0.068605,...,0.00033583961579626,0.021910424890855602,0.013159666615565323,0.06573127467256681,0.10899510118069831,0.8286341290830657,0.06037779608020233,0.03374344610612471,0.0331663107947236,0.18113338824060699
1360,918-a_n.wav,male,60+,pathological,0.784509,-0.094563,-0.508726,-0.114747,-0.688131,0.243823,...,0.00013198489228337428,0.013590239835238735,0.015650738949576547,0.0407707195057162,0.18714052268793396,1.6271621978605206,0.09309174114492928,0.11947836751956864,0.14660736998049834,0.27927522343478783
1361,922-a_n.wav,male,60+,pathological,-0.452221,0.285549,0.573179,-0.009584,-0.174004,0.073117,...,7.038792492946143e-05,0.005371658854774559,0.005195692957354728,0.016114976564323678,0.02510366256529753,0.20821059138465564,0.013297933248449958,0.015473906892882241,0.02119041066612836,0.03989379974534987
1362,923-a_n.wav,male,60+,pathological,-0.057950,0.317345,0.036295,0.060022,0.086797,0.132781,...,3.089923423628047e-05,0.00130480767963893,0.0018012834964944752,0.00391442303891679,0.01152860959294848,0.10084041118572165,0.004850377703462086,0.0073803224551516056,0.014499602944520273,0.014551133110386257


id
gender
age
status
MFCC-1
MFCC-2
MFCC-3
MFCC-4
MFCC-5
MFCC-6
MFCC-7
MFCC-8
MFCC-9
MFCC-10
MFCC-11
MFCC-12
MFCC-13
1st-derivative
2nd-derivative
meanF0Hz
stdevF0Hz
HNR
localJitter
localabsoluteJitter
rapJitter
ppq5Jitter
ddpJitter
localShimmer
localdbShimmer
apq3Shimmer
apq5Shimmer
apq11Shimmer
ddaShimmer


# Save DataFrame

In [157]:
f.to_csv('features.csv')